### Import data

In [ ]:
import os
import subprocess
from datetime import datetime, timedelta
from glob import glob
import geopandas as gpd
import numpy as np
import pandas as pd
import pyproj
from herbie import FastHerbie
from shapely import geometry as shpg
from shapely.ops import transform

In [ ]:
import pandas as pd

filepathmulti = "C:\\MULTIHFT_Extract.txt"
filepathsingle = "C:\\SINGLEHFT_Extract.txt"

df1 = pd.read_csv(filepathsingle)
df2 = pd.read_csv(filepathmulti)
df1.columns = ['OID_', 'pointid', 'grid_code', 'WHP', 'RPS',
       'NatForest', 'FLEP8', 'FLEP4', 'CRPS',
       'CFL', 'BP', 'StreamEuc', 'Slope',
       'RoadEuc', 'DEM', 'Aspect', 'StrucEuc',
       'NVC', 'PRS', 'SCLASS', 'VCC', 'PMS',
       'PLS', 'FRG', 'ACTIVITY_C']

df1.loc[(df1.PLS > 20), 'PLS'] = 0
df1.loc[(df1.PRS > 20), 'PRS'] = 0
df1.loc[(df1.PMS > 20), 'PMS'] = 0

df2.columns = ['OID_', 'pointid', 'grid_code', 'WHP', 'RPS',
       'NatForest', 'FLEP8', 'FLEP4', 'CRPS',
       'CFL', 'BP', 'StreamEuc', 'Slope',
       'RoadEuc', 'DEM', 'Aspect', 'StrucEuc',
       'NVC', 'PRS', 'SCLASS', 'VCC', 'PMS',
       'PLS', 'FRG', 'ACTIVITY_C']

df2.loc[(df2.ACTIVITY_C > 0), 'ACTIVITY_C'] = 1999
df2.loc[(df2.PLS > 20), 'PLS'] = 0
df2.loc[(df2.PRS > 20), 'PRS'] = 0
df2.loc[(df2.PMS > 20), 'PMS'] = 0

df = pd.concat([df1, df2], ignore_index=True).dropna()

print("The number samples", len(df))
df.columns.values

df = df.iloc[::24, :]
len(df)

In [ ]:
dfBBurn = df[df['ACTIVITY_C']==1111]
dfBurnPiled = df[df['ACTIVITY_C']==1130]
dfPiling = df[df['ACTIVITY_C']==1153]
dfYarding = df[df['ACTIVITY_C']==1120]
dfThinningHFR = df[df['ACTIVITY_C']==1160]
dfMULTI = df[df['ACTIVITY_C']==1999]

print('Broadcast burn           ', len(dfBBurn), len(dfBBurn)*.7,len(dfBBurn)*.3)
print('Burning of Piled Material', len(dfBurnPiled), len(dfBurnPiled)*.7,len(dfBurnPiled)*.3)
print('Piling of fuels          ', len(dfPiling), len(dfPiling)*.7,len(dfPiling)*.3)
print('Yarding                  ', len(dfYarding), len(dfYarding)*.7,len(dfYarding)*.3)
print('Thinning for HFR         ', len(dfThinningHFR), len(dfThinningHFR)*.7,len(dfThinningHFR)*.3)
print('Multi-Treatment         ', len(dfMULTI), len(dfMULTI)*.7,len(dfMULTI)*.3)

### Variance Inflation Factor

In [ ]:
# calculate VIF

from patsy import dmatrices
from statsmodels.stats.outliers_influence import variance_inflation_factor

#find design matrix for linear regression model using 'ACTIVITY_C' as response variable 
z, W = dmatrices('ACTIVITY_C ~ WHP+BP+FLEP8+FLEP4+CRPS+CFL+StreamEuc+Slope+RoadEuc+DEM+StrucEuc+PRS+PMS+PLS', 
                 data=df, 
                 return_type='dataframe')


#calculate VIF for each explanatory variable
vif = pd.DataFrame()
vif['VIF'] = [variance_inflation_factor(W.values, i) 
              for i in range(W.shape[1])]
vif['variable'] = W.columns

#view VIF for each explanatory variable 
vif

### Dataset statistics

In [ ]:

import statistics 

print("WHP          - min", min(df['WHP']), "max", max(df['WHP']), "Mean", statistics.mean(df['WHP']),"SD", statistics.stdev(df['WHP']))
print("BP           - min", min(df['BP']), "max", max(df['BP']), "Mean", statistics.mean(df['BP']),"SD", statistics.stdev(df['BP']))
print("FLEP8        - min", min(df['FLEP8']), "max", max(df['FLEP8']), "Mean", statistics.mean(df['FLEP8']),"SD", statistics.stdev(df['FLEP8']))
print("FLEP4        - min", min(df['FLEP4']), "max", max(df['FLEP4']), "Mean", statistics.mean(df['FLEP4']),"SD", statistics.stdev(df['FLEP4']))
print("CRPS         - min", min(df['CRPS']), "max", max(df['CRPS']), "Mean", statistics.mean(df['CRPS']),"SD", statistics.stdev(df['CRPS']))
print("CFL          - min", min(df['CFL']), "max", max(df['CFL']), "Mean", statistics.mean(df['CFL']),"SD", statistics.stdev(df['CFL']))
print("StreamEuc    - min", min(df['StreamEuc']), "max", max(df['StreamEuc']), "Mean", statistics.mean(df['StreamEuc']),"SD", statistics.stdev(df['StreamEuc']))
print("Slope        - min", min(df['Slope']), "max", max(df['Slope']), "Mean", statistics.mean(df['Slope']),"SD", statistics.stdev(df['Slope']))
print("RoadEuc      - min", min(df['RoadEuc']), "max", max(df['RoadEuc']), "Mean", statistics.mean(df['RoadEuc']),"SD", statistics.stdev(df['RoadEuc']))
print("DEM          - min", min(df['DEM']), "max", max(df['DEM']), "Mean", statistics.mean(df['DEM']),"SD", statistics.stdev(df['DEM']))
print("Aspect       - min", min(df['Aspect']), "max", max(df['Aspect']), "Mean", statistics.mean(df['Aspect']),"SD", statistics.stdev(df['Aspect']))
print("StrucEuc     - min", min(df['StrucEuc']), "max", max(df['StrucEuc']), "Mean", statistics.mean(df['StrucEuc']),"SD", statistics.stdev(df['StrucEuc']))
print("PRS          - min", min(df['PRS']), "max", max(df['PRS']), "Mean", statistics.mean(df['PRS']),"SD", statistics.stdev(df['PRS']))
print("PMS          - min", min(df['PMS']), "max", max(df['PMS']), "Mean", statistics.mean(df['PMS']),"SD", statistics.stdev(df['PMS']))
print("PLS          - min", min(df['PLS']), "max", max(df['PLS']), "Mean", statistics.mean(df['PLS']),"SD", statistics.stdev(df['PLS']))


### IMPORT National Forest data for Prediction

In [ ]:
import pandas as pd

filepathpred = "C:\\HFT_ForPrediction_90m.txt"

dfPrediction = pd.read_csv(filepathpred)

dfPrediction.columns = ['OID_', 'pointid', 'grid_code', 'WHP', 'RPS',
       'FLEP8', 'FLEP4', 'CRPS', 'CFL',
       'BP', 'StreamEuc', 'Slope', 'RoadEuc',
       'DEM', 'Aspect', 'StrucEuc', 'PRS',
       'SCLASS', 'VCC', 'PMS', 'PLS',
       'FRG', 'NatForest', 'Long', 'Lat']


### Statistics for NATIONAL FOREST DATASET

In [ ]:
import statistics 

print("WHP          - min", min(dfPrediction['WHP']), "max", max(dfPrediction['WHP']), "Mean", statistics.mean(dfPrediction['WHP']),"SD", statistics.stdev(dfPrediction['WHP']))
print("BP           - min", min(dfPrediction['BP']), "max", max(dfPrediction['BP']), "Mean", statistics.mean(dfPrediction['BP']),"SD", statistics.stdev(dfPrediction['BP']))
print("FLEP8        - min", min(dfPrediction['FLEP8']), "max", max(dfPrediction['FLEP8']), "Mean", statistics.mean(dfPrediction['FLEP8']),"SD", statistics.stdev(dfPrediction['FLEP8']))
print("FLEP4        - min", min(dfPrediction['FLEP4']), "max", max(dfPrediction['FLEP4']), "Mean", statistics.mean(dfPrediction['FLEP4']),"SD", statistics.stdev(dfPrediction['FLEP4']))
print("CRPS         - min", min(dfPrediction['CRPS']), "max", max(dfPrediction['CRPS']), "Mean", statistics.mean(dfPrediction['CRPS']),"SD", statistics.stdev(dfPrediction['CRPS']))
print("CFL          - min", min(dfPrediction['CFL']), "max", max(dfPrediction['CFL']), "Mean", statistics.mean(dfPrediction['CFL']),"SD", statistics.stdev(dfPrediction['CFL']))
print("StreamEuc    - min", min(dfPrediction['StreamEuc']), "max", max(dfPrediction['StreamEuc']), "Mean", statistics.mean(dfPrediction['StreamEuc']),"SD", statistics.stdev(dfPrediction['StreamEuc']))
print("Slope        - min", min(dfPrediction['Slope']), "max", max(dfPrediction['Slope']), "Mean", statistics.mean(dfPrediction['Slope']),"SD", statistics.stdev(dfPrediction['Slope']))
print("RoadEuc      - min", min(dfPrediction['RoadEuc']), "max", max(dfPrediction['RoadEuc']), "Mean", statistics.mean(dfPrediction['RoadEuc']),"SD", statistics.stdev(dfPrediction['RoadEuc']))
print("DEM          - min", min(dfPrediction['DEM']), "max", max(dfPrediction['DEM']), "Mean", statistics.mean(dfPrediction['DEM']),"SD", statistics.stdev(dfPrediction['DEM']))
print("Aspect       - min", min(dfPrediction['Aspect']), "max", max(dfPrediction['Aspect']), "Mean", statistics.mean(dfPrediction['Aspect']),"SD", statistics.stdev(dfPrediction['Aspect']))
print("StrucEuc     - min", min(dfPrediction['StrucEuc']), "max", max(dfPrediction['StrucEuc']), "Mean", statistics.mean(dfPrediction['StrucEuc']),"SD", statistics.stdev(dfPrediction['StrucEuc']))
print("PRS          - min", min(dfPrediction['PRS']), "max", max(dfPrediction['PRS']), "Mean", statistics.mean(dfPrediction['PRS']),"SD", statistics.stdev(dfPrediction['PRS']))
print("PMS          - min", min(dfPrediction['PMS']), "max", max(dfPrediction['PMS']), "Mean", statistics.mean(dfPrediction['PMS']),"SD", statistics.stdev(dfPrediction['PMS']))
print("PLS          - min", min(dfPrediction['PLS']), "max", max(dfPrediction['PLS']), "Mean", statistics.mean(dfPrediction['PLS']),"SD", statistics.stdev(dfPrediction['PLS']))


### Min Max DUMMY DATA

In [ ]:
datadummy = { 'OID_':[1,2],
             'pointid':[1,2],
             'grid_code':[1,2],
             'WHP':[0,7817],
             'RPS':[0,1.178],
             'NatForest':[1,2],
             'FLEP8':[0,1],
             'FLEP4':[0,1],
             'CRPS':[0,99.8465195],
             'CFL':[0,35.4152641 ],
             'BP':[0,0.0133],
             'StreamEuc':[0,4590.392],
             'Slope':[0,73],
             'RoadEuc':[0,13165.5917969],
             'DEM':[1623,4387],
             'Aspect':[-1,359.72],
             'StrucEuc':[0,12844.09],
             'NVC': [1,2],
             'PRS': [2,20],
             'SCLASS': [1,2],
             'VCC': [1,2],
             'PMS':[1,16],
             'PLS':[1,18],
             'FRG':[1,2],
             'ACTIVITY_C':[1,2]}
datadummy = pd.DataFrame(datadummy)
datadummy

df = df.append([datadummy], ignore_index = True).dropna()


### Data Prep and Data min-max and OHE

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split

feature_cols = ['FLEP8', 'FLEP4','WHP','BP', 'StreamEuc', 'Slope', 'RoadEuc', 'StrucEuc', 'DEM', 'PRS', 'PMS', 'PLS']

X = df.loc[:, feature_cols]
y1 = df.iloc[:, 24:25].values
y_lab = df.iloc[:, 24].values

X_train, X_test, y_train, y_test = train_test_split(X, y1, test_size=0.30)
sc = MinMaxScaler(feature_range=(0, 1))
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
ohe = OneHotEncoder()
y_train = ohe.fit_transform(y_train).toarray()
y_test = ohe.transform(y_test).toarray()

print(X_train[1])
print(y_train[1])

print("number of samples for training", len(X_train) + len(X_test))
print("number of samples for training", len(X_train))
print("number of samples for validation", len(X_test))


### BOXPLOT OF TRANSFORMED VARIABLES

In [ ]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0,1))

X = df.loc[:, ['FLEP8', 'FLEP4','WHP','BP',
       'StreamEuc', 'Slope', 'RoadEuc', 'StrucEuc',
        'DEM', 'PRS','PMS','PLS','ACTIVITY_C','grid_code']]#, 'CO_Aspect'
X = sc.fit_transform(X)
X

# convert numpy array to dataframe
dfB = pd.DataFrame(X, columns =['FLEP8', 'FLEP4','WHP','BP',
       'StreamEuc', 'Slope', 'RoadEuc', 'StrucEuc',
        'DEM', 'PRS','PMS','PLS','ACTIVITY_C', 'grid_code'])
dfB

In [ ]:
import numpy
numpy.unique(dfB["ACTIVITY_C"])

dfB.ACTIVITY_C = dfB.ACTIVITY_C.round(4)

In [ ]:
dfB.loc[(dfB['ACTIVITY_C'] == 0.0), 'ACTIVITY'] = 'Broadcast Burning'
dfB.loc[(dfB['ACTIVITY_C'] == 0.0101) , 'ACTIVITY'] = 'Yarding'
dfB.loc[(dfB['ACTIVITY_C'] == 0.0214), 'ACTIVITY'] = 'Burning of Piled Material'
dfB.loc[(dfB['ACTIVITY_C'] == 0.0473), 'ACTIVITY'] = 'Piling of Fuels'
dfB.loc[(dfB['ACTIVITY_C'] == 0.0552), 'ACTIVITY'] = 'Thinning for Hazardous Fuels Reduction'
dfB.loc[(dfB['ACTIVITY_C'] == 1.0), 'ACTIVITY'] = 'Multi-Treatment'

In [ ]:
dflong = dfB.reset_index()
dflong = pd.melt(dflong, id_vars='ACTIVITY', value_vars=['FLEP8', 'FLEP4','WHP','BP',
       'StreamEuc', 'Slope', 'RoadEuc', 'StrucEuc',
        'DEM', 'PRS','PMS','PLS'])

In [ ]:
dflong = dflong.rename(columns={"ACTIVITY":"Activity"})
dflong = dflong.rename(columns={"value":"Transformed Value"})
dflong = dflong.rename(columns={"variable":"Variable"})

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.set(rc={'figure.figsize':(15.7,8.27)},style="whitegrid",font_scale=1.2)

hue_order = ['Broadcast Burning', 'Yarding', 'Burning of Piled Material','Piling of Fuels','Thinning for Hazardous Fuels Reduction', 'Multi-Treatment']


sns.boxplot(x = 'Variable',
            y = 'Transformed Value',
            hue = 'Activity',
            hue_order=hue_order,
            palette = 'viridis',
            data =dflong,
            fliersize =2)

plt.legend(loc='upper center', title='Treatment Type')
plt.savefig("C:\\HFT\\Figures\\BoxPlot.png", transparent=False,dpi=300, bbox_inches = "tight",facecolor='white')#


In [ ]:
W = df.loc[:, ['FLEP8', 'FLEP4','WHP','BP',
       'StreamEuc', 'Slope', 'RoadEuc', 'StrucEuc',
        'DEM', 'PRS','PMS','PLS']]
W = sc.fit_transform(W)
W
dataframe=pd.DataFrame(W)

In [ ]:
dataframe2 = dataframe.drop_duplicates( subset=[0, 1, 9,10], keep='last')

### Build, compile, fit model

In [ ]:
import keras
from keras import Sequential 
from keras.layers import Dense
from tensorflow.keras.optimizers import Adam

model = Sequential()
model.add(Dense(64, activation = "relu"))
model.add(Dense(6, activation = "softmax"))
model.compile(loss = "categorical_crossentropy",
             optimizer = Adam(learning_rate=0.001),
             metrics = ["accuracy"])

history = model.fit(X_train, y_train, epochs = 25, batch_size = 64, validation_data=(X_test, y_test))

### SAVE MODEL

In [ ]:
#model.save("C:\\Users\\Desktop\\HFT\\HFT_KerasModel.h5")

In [ ]:
# load and evaluate a saved model
from numpy import loadtxt
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam

# load model
model = load_model("C:\\Users\\Desktop\\HFT\\HFT_KerasModel_1_9_2023.h5")

model.compile(loss = "categorical_crossentropy",
             optimizer = Adam(learning_rate=0.001),
             metrics = ["accuracy"])

fit=model.fit(X_train, y_train)

In [ ]:
# Make predictions on the validation data
y_pred = model.predict(X_test)

# Create a PDPbox instance
from pdpbox.pdp import pdp_isolate, pdp_plot

# Generate the PDP for the 'FLEP8' feature
feature_name = 'FLEP8'
pdp_flep8 = pdp_isolate(
                model=model, 
                dataset=X_test, 
                model_features=X_train.columns, 
                feature=feature_name
            )

# Plot the PDP for the 'FLEP8' feature
pdp_plot(pdp_flep8, feature_name)

In [ ]:
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.inspection import PartialDependenceDisplay

X, y = make_hastie_10_2(random_state=10)
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
    max_depth=1, random_state=0).fit(X, y)
features = [0, 1, (0, 1)]
PartialDependenceDisplay.from_estimator(clf, X, features)

In [ ]:
import pandas as pd
from pdpbox import pdp, info_plots
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier # or any other model

feature = 'feature_name'  # Replace 'feature_name' with the name of the feature you're interested in
pdp_dist = pdp.pdp_isolate(model=model, dataset=X_train, model_features=X_train.columns, feature=feature)
pdp.pdp_plot(pdp_dist, feature)


In [ ]:
import numpy as np
from sklearn.inspection import partial_dependence
import pdpbox
from tensorflow.keras.models import load_model

# define custom function to wrap Keras model predict method
def predict_fn(X):
    return model.predict(X).ravel()

# define the feature index and name
feature_idx = 0  # replace with the index of the feature of interest
feature_name = "feature_name"  # replace with the name of the feature of interest

# extract the values of the feature from the input data
feature_values = X[:, feature_idx]

# compute the partial dependence
pdp, axes = pdpbox.pdp.pdp_isolate(model=model,
                                    dataset=X,
                                    model_features=model_features,
                                    feature=feature_name,
                                    feature_grids=[feature_values],
                                    num_grid_points=50,
                                    predict_fn=predict_fn,
                                    n_jobs=-1)

# plot the partial dependence
fig, axes = pdpbox.pdp.pdp_plot(pdp, feature_name, plot_lines=True, frac_to_plot=0.5, plot_pts_dist=True)


In [ ]:
import numpy as np
from sklearn.inspection import partial_dependence
from sklearn.inspection import plot_partial_dependence
import matplotlib.pyplot as plt

def predict_fn(X):
    return model.predict(X)

feature_idx = 0  # replace with the index of the feature of interest

pdp_results = partial_dependence(model, X_train, [feature_idx], method='brute', predict=predict_fn)

plt.figure()
plot_partial_dependence(pdp_results, X_train, [feature_idx])
plt.xlabel(f"Feature {feature_idx}")
plt.ylabel("Predicted Probability")
plt.show()


In [ ]:
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.inspection import PartialDependenceDisplay

def predict_fn(X_test):
    return model.predict(X_test).ravel()

features_to_plot = [(0, 1), 2]  # features to plot

PartialDependenceDisplay.from_estimator(predict_fn, X_test, features)

plotPartialDependence(model,1)

In [ ]:
from sklearn.inspection import PartialDependenceDisplay
from sklearn.datasets import make_friedman1
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(12, 6))
ax.set_title("Decision Tree")
tree_disp = PartialDependenceDisplay.from_estimator(fit, X_train, [0, 0])

In [ ]:
import shap
fig, ax = plt.subplots(figsize=(12, 6))
ax.set_title("Decision Tree")
tree_disp = PartialDependenceDisplay.from_estimator(tree, X, [1, 2], ax=ax)

In [ ]:
shap.plots.partial_dependence(
    [1], model.predict, X_train, ice=True,
    model_expected_value=True, feature_expected_value=True
)

In [ ]:
data = df

# extract input features and target variable
numerical_features = ['FLEP8', 'FLEP4','WHP','BP',
       'StreamEuc', 'Slope', 'RoadEuc', 'StrucEuc',
        'DEM', 'PRS','PMS','PLS']
#categorical_features = ['categorical_feature1', 'categorical_feature2']
target = 'ACTIVITY_C'
X = X_train
y = y_train

In [ ]:
from sklearn.inspection import PartialDependenceDisplay
from sklearn.inspection import plot_partial_dependence
from sklearn.datasets import load_boston
from sklearn.ensemble import GradientBoostingRegressor

n = 5
PartialDependenceDisplay.from_estimator(
    model, X_train, ['dem'], kind="both", n_jobs=3, subsample=n
)
plt.legend(bbox_to_anchor=(1,1));

In [ ]:
import matplotlib.pyplot as plt

for feature in numerical_features:
    # define the range of values for the feature
    feature_values = np.linspace(data[feature].min(), data[feature].max(), num=10)

    # calculate the PDP values for each feature value
    pdp_values = []
    for value in feature_values:
        X_temp = X.copy()
        X_temp[:, numerical_features.index(feature)] = value
        pdp_values.append(model.predict(X_temp).mean())

    # plot the PDP curve
    plt.plot(feature_values, pdp_values)
    plt.xlabel(feature)
    plt.ylabel('Predicted value')
    plt.show()
    


### Confusion Matrix

In [ ]:
import numpy as np
pred = model.predict(X)
pred = np.argmax(pred, axis = 1)[:]
label = np.argmax(y, axis = 1)[:]

print(pred)
print(label)
#df['predictionResult'] = pred.tolist()


### CONFUSION MATRIX ###########################################################
%matplotlib inline
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt
cm = confusion_matrix(y_true=label, y_pred=pred)

def plot_confusion_matrix(cm, classes,
                        normalize=False,
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):
    plt.figure(figsize = (15,15))
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

cm_plot_labels = ['Broadcast Burning','Yarding','Burning of Piled Material','Piling of Fuels','Thinning for HFR','Multi-Treatment']
plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title='Confusion Matrix')

### END CONFUSION MATRIX #######################################################

# AUC full and variable Isolation

### AUC - FULL

In [ ]:
from keras import models
from tensorflow.keras.optimizers import Adam
import keras
from keras import Sequential 
from keras.layers import Dense
from tensorflow.keras.optimizers import Adam

X = df.loc[:,['FLEP8','FLEP4','WHP','BP',
       'StreamEuc', 'Slope', 'RoadEuc', 'StrucEuc',
       'DEM', 'PRS','PMS','PLS']] 

y = df.iloc[:,24:25].values
y_lab = df.iloc[:,24].values


from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0,1))
X = sc.fit_transform(X)

from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
y = ohe.fit_transform(y).toarray()

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.33)

model = Sequential()
model.add(Dense(64, activation = "relu"))
model.add(Dense(6, activation = "softmax"))
model.compile(loss = "binary_crossentropy",
             optimizer = Adam(learning_rate=0.001),
             metrics = ["accuracy"])

history = model.fit(X_train, y_train, epochs = 1, batch_size = 64, validation_data=(X_test, y_test))



y_val_cat_prob=model.predict(X_test)
n_classes=6
import numpy as np
import matplotlib.pyplot as plt
from itertools import cycle
from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_auc_score

fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve( y_test[:, i] , y_val_cat_prob[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])    
    

lw=.8
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])

mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

plt.figure()

plt.plot(
    fpr["macro"],
    tpr["macro"],
    label="macro-average ROC curve (area = {0:0.2f})".format(roc_auc["macro"]),
    color="navy",
    linestyle=":",
    linewidth=4,
)

label = ["1","2","3","4","5","6"]

colors = cycle(["aqua", "darkorange", "cornflowerblue"])
for i, color in zip(range(n_classes), colors):
    plt.plot(
        fpr[i],
        tpr[i],
        color=color,
        lw=lw,
        label="ROC curve of class {0} (area = {1:0.2f})".format(i, roc_auc[i]),
    )

plt.plot([0, 1], [0, 1], "k--", lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Some extension of Receiver operating characteristic to multiclass")
plt.legend(loc="lower right")
plt.show()

MACROFULL = roc_auc["macro"]
MACROFULL0 = roc_auc[0]
MACROFULL1 = roc_auc[1]
MACROFULL2 = roc_auc[2]
MACROFULL3 = roc_auc[3]
MACROFULL4 = roc_auc[4]
MACROFULL5 = roc_auc[5]

print(MACROFULL)


### 'FLEP8'

In [ ]:
from keras import models
from tensorflow.keras.optimizers import Adam

X = df.loc[:,['FLEP4','WHP','BP',
       'StreamEuc', 'Slope', 'RoadEuc', 'StrucEuc',
        'DEM', 'PRS','PMS','PLS']]

y = df.iloc[:,24:25].values
y_lab = df.iloc[:,24].values

from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0,1))
X = sc.fit_transform(X)

from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
y = ohe.fit_transform(y).toarray()

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.33)

model = Sequential()
model.add(Dense(64, activation = "relu"))
model.add(Dense(6, activation = "softmax"))
model.compile(loss = "binary_crossentropy",
             optimizer = Adam(learning_rate=0.001),
             metrics = ["accuracy"])

history = model.fit(X_train, y_train, epochs = 1, batch_size = 64, validation_data=(X_test, y_test))

y_val_cat_prob=model.predict(X_test)
n_classes=6
import numpy as np
import matplotlib.pyplot as plt
from itertools import cycle
from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_auc_score

fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve( y_test[:, i] , y_val_cat_prob[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])    
    

lw=.8
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])

mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

plt.figure()

plt.plot(
    fpr["macro"],
    tpr["macro"],
    label="macro-average ROC curve (area = {0:0.2f})".format(roc_auc["macro"]),
    color="navy",
    linestyle=":",
    linewidth=4,
)

label = ["1","2","3","4","5","6"]

colors = cycle(["aqua", "darkorange", "cornflowerblue"])
for i, color in zip(range(n_classes), colors):
    plt.plot(
        fpr[i],
        tpr[i],
        color=color,
        lw=lw,
        label="ROC curve of class {0} (area = {1:0.2f})".format(i, roc_auc[i]),
    )

plt.plot([0, 1], [0, 1], "k--", lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Some extension of Receiver operating characteristic to multiclass")
plt.legend(loc="lower right")
plt.show()

MACROFLEP8 = roc_auc["macro"]
MACROFLEP80 = roc_auc[0]
MACROFLEP81 = roc_auc[1]
MACROFLEP82 = roc_auc[2]
MACROFLEP83 = roc_auc[3]
MACROFLEP84 = roc_auc[4]
MACROFLEP85 = roc_auc[5]

print(MACROFLEP8)

### 'FLEP4'

In [ ]:
from keras import models
from tensorflow.keras.optimizers import Adam
import keras
from keras import Sequential 
from keras.layers import Dense
from tensorflow.keras.optimizers import Adam


X = df.loc[:,['FLEP8','WHP','BP',
       'StreamEuc', 'Slope', 'RoadEuc', 'StrucEuc',
        'DEM', 'PRS','PMS','PLS']]

y = df.iloc[:,24:25].values
y_lab = df.iloc[:,24].values

from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0,1))
X = sc.fit_transform(X)

from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
y = ohe.fit_transform(y).toarray()

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.33)


model = Sequential()
model.add(Dense(64, activation = "relu"))
model.add(Dense(6, activation = "softmax"))
model.compile(loss = "binary_crossentropy",
             optimizer = Adam(learning_rate=0.001),
             metrics = ["accuracy"])

history = model.fit(X_train, y_train, epochs = 1, batch_size = 64, validation_data=(X_test, y_test))



y_val_cat_prob=model.predict(X_test)
n_classes=6

import numpy as np
import matplotlib.pyplot as plt
from itertools import cycle
from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_auc_score

fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve( y_test[:, i] , y_val_cat_prob[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])    
    

lw=.8
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])

mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])



MACROFLEP4 = roc_auc["macro"]
MACROFLEP40 = roc_auc[0]
MACROFLEP41 = roc_auc[1]
MACROFLEP42 = roc_auc[2]
MACROFLEP43 = roc_auc[3]
MACROFLEP44 = roc_auc[4]
MACROFLEP45 = roc_auc[5]

print(MACROFLEP4)

### 'WHP'

In [ ]:
from keras import models
from tensorflow.keras.optimizers import Adam
import keras
from keras import Sequential 
from keras.layers import Dense
from tensorflow.keras.optimizers import Adam

# whp
X = df.loc[:,['FLEP8','FLEP4','BP',
       'StreamEuc', 'Slope', 'RoadEuc', 'StrucEuc',
        'DEM', 'PRS','PMS','PLS']]

y = df.iloc[:,24:25].values
y_lab = df.iloc[:,24].values

from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0,1))
X = sc.fit_transform(X)

from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
y = ohe.fit_transform(y).toarray()

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.33)


model = Sequential()
model.add(Dense(64, activation = "relu"))
model.add(Dense(6, activation = "softmax"))
model.compile(loss = "binary_crossentropy",
             optimizer = Adam(learning_rate=0.001),
             metrics = ["accuracy"])

history = model.fit(X_train, y_train, epochs = 1, batch_size = 64, validation_data=(X_test, y_test))

y_val_cat_prob=model.predict(X_test)
n_classes=6


fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve( y_test[:, i] , y_val_cat_prob[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])    

lw=.8
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])

mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])



MACROWHP = roc_auc["macro"]
MACROWHP0 = roc_auc[0]
MACROWHP1 = roc_auc[1]
MACROWHP2 = roc_auc[2]
MACROWHP3 = roc_auc[3]
MACROWHP4 = roc_auc[4]
MACROWHP5 = roc_auc[5]

print(MACROWHP)

### 'BP'

In [ ]:
from keras import models
from tensorflow.keras.optimizers import Adam

X = df.loc[:,['FLEP8','FLEP4','WHP',
       'StreamEuc', 'Slope', 'RoadEuc', 'StrucEuc',
        'DEM', 'PRS','PMS','PLS']]

y = df.iloc[:,24:25].values
y_lab = df.iloc[:,24].values


from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0,1))
X = sc.fit_transform(X)

from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
y = ohe.fit_transform(y).toarray()

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.33)

scores = model.evaluate(X_test, y_test, verbose=False)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

model = Sequential()
model.add(Dense(64, activation = "relu"))
model.add(Dense(6, activation = "softmax"))
model.compile(loss = "binary_crossentropy",
             optimizer = Adam(learning_rate=0.001),
             metrics = ["accuracy"])

history = model.fit(X_train, y_train, epochs = 1, batch_size = 64, validation_data=(X_test, y_test))

y_val_cat_prob=model.predict(X_test)
n_classes=6
import numpy as np
import matplotlib.pyplot as plt
from itertools import cycle
from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_auc_score

fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve( y_test[:, i] , y_val_cat_prob[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])    

lw=.8
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])

mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

plt.figure()


plt.plot(
    fpr["macro"],
    tpr["macro"],
    label="macro-average ROC curve (area = {0:0.2f})".format(roc_auc["macro"]),
    color="navy",
    linestyle=":",
    linewidth=4,
)

label = ["1","2","3","4","5","6"]

colors = cycle(["aqua", "darkorange", "cornflowerblue"])
for i, color in zip(range(n_classes), colors):
    plt.plot(
        fpr[i],
        tpr[i],
        color=color,
        lw=lw,
        label="ROC curve of class {0} (area = {1:0.2f})".format(i, roc_auc[i]),
    )

plt.plot([0, 1], [0, 1], "k--", lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Some extension of Receiver operating characteristic to multiclass")
plt.legend(loc="lower right")
plt.show()

MACROBP = roc_auc["macro"]
MACROBP0 = roc_auc[0]
MACROBP1 = roc_auc[1]
MACROBP2 = roc_auc[2]
MACROBP3 = roc_auc[3]
MACROBP4 = roc_auc[4]
MACROBP5 = roc_auc[5]

print(MACROBP)


### 'StreamEuc'

In [ ]:
from keras import models
from tensorflow.keras.optimizers import Adam

X = df.loc[:,['FLEP8','FLEP4','WHP','BP',
        'Slope', 'RoadEuc', 'StrucEuc',
        'DEM', 'PRS','PMS','PLS']]

y = df.iloc[:,24:25].values
y_lab = df.iloc[:,24].values

from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0,1))
X = sc.fit_transform(X)

from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
y = ohe.fit_transform(y).toarray()

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.33)

scores = model.evaluate(X_test, y_test, verbose=False)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


model = Sequential()
model.add(Dense(64, activation = "relu"))
model.add(Dense(6, activation = "softmax"))
model.compile(loss = "binary_crossentropy",
             optimizer = Adam(learning_rate=0.001),
             metrics = ["accuracy"])

history = model.fit(X_train, y_train, epochs = 1, batch_size = 64, validation_data=(X_test, y_test))

y_val_cat_prob=model.predict(X_test)
n_classes=6
import numpy as np
import matplotlib.pyplot as plt
from itertools import cycle
from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_auc_score

fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve( y_test[:, i] , y_val_cat_prob[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])    

lw=.8
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])

mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

plt.figure()


plt.plot(
    fpr["macro"],
    tpr["macro"],
    label="macro-average ROC curve (area = {0:0.2f})".format(roc_auc["macro"]),
    color="navy",
    linestyle=":",
    linewidth=4,
)

label = ["1","2","3","4","5","6"]

colors = cycle(["aqua", "darkorange", "cornflowerblue"])
for i, color in zip(range(n_classes), colors):
    plt.plot(
        fpr[i],
        tpr[i],
        color=color,
        lw=lw,
        label="ROC curve of class {0} (area = {1:0.2f})".format(i, roc_auc[i]),
    )

plt.plot([0, 1], [0, 1], "k--", lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Some extension of Receiver operating characteristic to multiclass")
plt.legend(loc="lower right")
plt.show()

MACROStreamEuc = roc_auc["macro"]
MACROStreamEuc0 = roc_auc[0]
MACROStreamEuc1 = roc_auc[1]
MACROStreamEuc2 = roc_auc[2]
MACROStreamEuc3 = roc_auc[3]
MACROStreamEuc4 = roc_auc[4]
MACROStreamEuc5 = roc_auc[5]

print(MACROStreamEuc)


### 'Slope'

In [ ]:
from keras import models
from tensorflow.keras.optimizers import Adam

X = df.loc[:,['FLEP8','FLEP4','WHP','BP',
       'StreamEuc', 'RoadEuc', 'StrucEuc',
        'DEM', 'PRS','PMS','PLS']]

y = df.iloc[:,24:25].values
y_lab = df.iloc[:,24].values

from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0,1))
X = sc.fit_transform(X)

from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
y = ohe.fit_transform(y).toarray()

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.33)

scores = model.evaluate(X_test, y_test, verbose=False)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


model = Sequential()
model.add(Dense(64, activation = "relu"))
model.add(Dense(6, activation = "softmax"))
model.compile(loss = "binary_crossentropy",
             optimizer = Adam(learning_rate=0.001),
             metrics = ["accuracy"])

history = model.fit(X_train, y_train, epochs = 1, batch_size = 64, validation_data=(X_test, y_test))



y_val_cat_prob=model.predict(X_test)
n_classes=6
import numpy as np
import matplotlib.pyplot as plt
from itertools import cycle
from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_auc_score

fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve( y_test[:, i] , y_val_cat_prob[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])    
    

lw=.8
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])

mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

plt.figure()

plt.plot(
    fpr["macro"],
    tpr["macro"],
    label="macro-average ROC curve (area = {0:0.2f})".format(roc_auc["macro"]),
    color="navy",
    linestyle=":",
    linewidth=4,
)

label = ["1","2","3","4","5","6"]

colors = cycle(["aqua", "darkorange", "cornflowerblue"])
for i, color in zip(range(n_classes), colors):
    plt.plot(
        fpr[i],
        tpr[i],
        color=color,
        lw=lw,
        label="ROC curve of class {0} (area = {1:0.2f})".format(i, roc_auc[i]),
    )

plt.plot([0, 1], [0, 1], "k--", lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Some extension of Receiver operating characteristic to multiclass")
plt.legend(loc="lower right")
plt.show()

MACROSlope = roc_auc["macro"]
MACROSlope0 = roc_auc[0]
MACROSlope1 = roc_auc[1]
MACROSlope2 = roc_auc[2]
MACROSlope3 = roc_auc[3]
MACROSlope4 = roc_auc[4]
MACROSlope5 = roc_auc[5]

print(MACROSlope)


### 'RoadEuc'

In [ ]:
from keras import models
from tensorflow.keras.optimizers import Adam

X = df.loc[:,['FLEP8','FLEP4','WHP','BP',
       'StreamEuc', 'Slope', 'StrucEuc',
        'DEM', 'PRS','PMS','PLS']]

y = df.iloc[:,24:25].values
y_lab = df.iloc[:,24].values

from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0,1))
X = sc.fit_transform(X)

from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
y = ohe.fit_transform(y).toarray()

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.33)

scores = model.evaluate(X_test, y_test, verbose=False)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

model = Sequential()
model.add(Dense(64, activation = "relu"))
model.add(Dense(6, activation = "softmax"))
model.compile(loss = "binary_crossentropy",
             optimizer = Adam(learning_rate=0.001),
             metrics = ["accuracy"])

history = model.fit(X_train, y_train, epochs = 1, batch_size = 64, validation_data=(X_test, y_test))

y_val_cat_prob=model.predict(X_test)
n_classes=6
import numpy as np
import matplotlib.pyplot as plt
from itertools import cycle
from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_auc_score

fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve( y_test[:, i] , y_val_cat_prob[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])    
    
lw=.8
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])

mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

plt.figure()


plt.plot(
    fpr["macro"],
    tpr["macro"],
    label="macro-average ROC curve (area = {0:0.2f})".format(roc_auc["macro"]),
    color="navy",
    linestyle=":",
    linewidth=4,
)

label = ["1","2","3","4","5","6"]

colors = cycle(["aqua", "darkorange", "cornflowerblue"])
for i, color in zip(range(n_classes), colors):
    plt.plot(
        fpr[i],
        tpr[i],
        color=color,
        lw=lw,
        label="ROC curve of class {0} (area = {1:0.2f})".format(i, roc_auc[i]),
        #label = label[i]
    )

plt.plot([0, 1], [0, 1], "k--", lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Some extension of Receiver operating characteristic to multiclass")
plt.legend(loc="lower right")
plt.show()

MACRORoadEuc = roc_auc["macro"]
MACRORoadEuc0 = roc_auc[0]
MACRORoadEuc1 = roc_auc[1]
MACRORoadEuc2 = roc_auc[2]
MACRORoadEuc3 = roc_auc[3]
MACRORoadEuc4 = roc_auc[4]
MACRORoadEuc5 = roc_auc[5]

print(MACRORoadEuc)


### 'StrucEuc'

In [ ]:
from keras import models
from tensorflow.keras.optimizers import Adam


X = df.loc[:,['FLEP8','FLEP4','WHP','BP',
       'StreamEuc', 'Slope', 'RoadEuc',
        'DEM', 'PRS','PMS','PLS']]

y = df.iloc[:,24:25].values
y_lab = df.iloc[:,24].values


from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0,1))
X = sc.fit_transform(X)

from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
y = ohe.fit_transform(y).toarray()

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.33)

scores = model.evaluate(X_test, y_test, verbose=False)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


model = Sequential()
model.add(Dense(64, activation = "relu"))
model.add(Dense(6, activation = "softmax"))
model.compile(loss = "binary_crossentropy",
             optimizer = Adam(learning_rate=0.001),
             metrics = ["accuracy"])

history = model.fit(X_train, y_train, epochs = 1, batch_size = 64, validation_data=(X_test, y_test))



y_val_cat_prob=model.predict(X_test)
n_classes=6
import numpy as np
import matplotlib.pyplot as plt
from itertools import cycle
from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_auc_score

fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve( y_test[:, i] , y_val_cat_prob[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])    

lw=.8
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])

mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# Plot all ROC curves
plt.figure()


plt.plot(
    fpr["macro"],
    tpr["macro"],
    label="macro-average ROC curve (area = {0:0.2f})".format(roc_auc["macro"]),
    color="navy",
    linestyle=":",
    linewidth=4,
)

label = ["1","2","3","4","5","6"]

colors = cycle(["aqua", "darkorange", "cornflowerblue"])
for i, color in zip(range(n_classes), colors):
    plt.plot(
        fpr[i],
        tpr[i],
        color=color,
        lw=lw,
        label="ROC curve of class {0} (area = {1:0.2f})".format(i, roc_auc[i]),
        #label = label[i]
    )

plt.plot([0, 1], [0, 1], "k--", lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Some extension of Receiver operating characteristic to multiclass")
plt.legend(loc="lower right")
plt.show()

MACROStrucEuc = roc_auc["macro"]
MACROStrucEuc0 = roc_auc[0]
MACROStrucEuc1 = roc_auc[1]
MACROStrucEuc2 = roc_auc[2]
MACROStrucEuc3 = roc_auc[3]
MACROStrucEuc4 = roc_auc[4]
MACROStrucEuc5 = roc_auc[5]

print(MACROStrucEuc)


### 'DEM'

In [ ]:
from keras import models
from tensorflow.keras.optimizers import Adam


X = df.loc[:,['FLEP8','FLEP4','WHP','BP',
       'StreamEuc', 'Slope', 'RoadEuc', 'StrucEuc',
        'PRS','PMS','PLS']]

y = df.iloc[:,24:25].values
y_lab = df.iloc[:,24].values


from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0,1))
X = sc.fit_transform(X)

from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
y = ohe.fit_transform(y).toarray()

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.33)

scores = model.evaluate(X_test, y_test, verbose=False)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


model = Sequential()
model.add(Dense(64, activation = "relu"))
model.add(Dense(6, activation = "softmax"))
model.compile(loss = "binary_crossentropy",
             optimizer = Adam(learning_rate=0.001),
             metrics = ["accuracy"])

history = model.fit(X_train, y_train, epochs = 1, batch_size = 64, validation_data=(X_test, y_test))



y_val_cat_prob=model.predict(X_test)
n_classes=6
import numpy as np
import matplotlib.pyplot as plt
from itertools import cycle
from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_auc_score

fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve( y_test[:, i] , y_val_cat_prob[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])    
    
lw=.8
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# Plot all ROC curves
plt.figure()


plt.plot(
    fpr["macro"],
    tpr["macro"],
    label="macro-average ROC curve (area = {0:0.2f})".format(roc_auc["macro"]),
    color="navy",
    linestyle=":",
    linewidth=4,
)

label = ["1","2","3","4","5","6"]

colors = cycle(["aqua", "darkorange", "cornflowerblue"])
for i, color in zip(range(n_classes), colors):
    plt.plot(
        fpr[i],
        tpr[i],
        color=color,
        lw=lw,
        label="ROC curve of class {0} (area = {1:0.2f})".format(i, roc_auc[i]),
    )

plt.plot([0, 1], [0, 1], "k--", lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Some extension of Receiver operating characteristic to multiclass")
plt.legend(loc="lower right")
plt.show()

MACRODEM = roc_auc["macro"]
MACRODEM0 = roc_auc[0]
MACRODEM1 = roc_auc[1]
MACRODEM2 = roc_auc[2]
MACRODEM3 = roc_auc[3]
MACRODEM4 = roc_auc[4]
MACRODEM5 = roc_auc[5]

print(MACRODEM)


### 'PRS'

In [ ]:
from keras import models
from tensorflow.keras.optimizers import Adam

X = df.loc[:,['FLEP8','FLEP4','WHP','BP',
       'StreamEuc', 'Slope', 'RoadEuc', 'StrucEuc',
        'DEM','PMS','PLS']]

y = df.iloc[:,24:25].values
y_lab = df.iloc[:,24].values


from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0,1))
X = sc.fit_transform(X)

from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
y = ohe.fit_transform(y).toarray()

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.33)

scores = model.evaluate(X_test, y_test, verbose=False)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


model = Sequential()
model.add(Dense(64, activation = "relu"))
model.add(Dense(6, activation = "softmax"))
model.compile(loss = "binary_crossentropy",
             optimizer = Adam(learning_rate=0.001),
             metrics = ["accuracy"])

history = model.fit(X_train, y_train, epochs = 1, batch_size = 64, validation_data=(X_test, y_test))



y_val_cat_prob=model.predict(X_test)
n_classes=6
import numpy as np
import matplotlib.pyplot as plt
from itertools import cycle
from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_auc_score

fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve( y_test[:, i] , y_val_cat_prob[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])    
    
lw=.8
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])

mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

plt.figure()

plt.plot(
    fpr["macro"],
    tpr["macro"],
    label="macro-average ROC curve (area = {0:0.2f})".format(roc_auc["macro"]),
    color="navy",
    linestyle=":",
    linewidth=4,
)

label = ["1","2","3","4","5","6"]

colors = cycle(["aqua", "darkorange", "cornflowerblue"])
for i, color in zip(range(n_classes), colors):
    plt.plot(
        fpr[i],
        tpr[i],
        color=color,
        lw=lw,
        label="ROC curve of class {0} (area = {1:0.2f})".format(i, roc_auc[i]),
        #label = label[i]
    )

plt.plot([0, 1], [0, 1], "k--", lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Some extension of Receiver operating characteristic to multiclass")
plt.legend(loc="lower right")
plt.show()

MACROPRS = roc_auc["macro"]
MACROPRS0 = roc_auc[0]
MACROPRS1 = roc_auc[1]
MACROPRS2 = roc_auc[2]
MACROPRS3 = roc_auc[3]
MACROPRS4 = roc_auc[4]
MACROPRS5 = roc_auc[5]

print(MACROPRS)


### 'PMS'

In [ ]:
from keras import models
from tensorflow.keras.optimizers import Adam

X = df.loc[:,['FLEP8','FLEP4','WHP','BP',
       'StreamEuc', 'Slope', 'RoadEuc', 'StrucEuc',
        'DEM', 'PRS','PLS']]

y = df.iloc[:,24:25].values
y_lab = df.iloc[:,24].values


from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0,1))
X = sc.fit_transform(X)

from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
y = ohe.fit_transform(y).toarray()

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.33)

scores = model.evaluate(X_test, y_test, verbose=False)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


model = Sequential()
model.add(Dense(64, activation = "relu"))
model.add(Dense(6, activation = "softmax"))
model.compile(loss = "binary_crossentropy",
             optimizer = Adam(learning_rate=0.001),
             metrics = ["accuracy"])

history = model.fit(X_train, y_train, epochs = 1, batch_size = 64, validation_data=(X_test, y_test))



y_val_cat_prob=model.predict(X_test)
n_classes=6
import numpy as np
import matplotlib.pyplot as plt
from itertools import cycle
from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_auc_score

fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve( y_test[:, i] , y_val_cat_prob[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])    
    
lw=.8

all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])

mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

plt.figure()

plt.plot(
    fpr["macro"],
    tpr["macro"],
    label="macro-average ROC curve (area = {0:0.2f})".format(roc_auc["macro"]),
    color="navy",
    linestyle=":",
    linewidth=4,
)

label = ["1","2","3","4","5","6"]

colors = cycle(["aqua", "darkorange", "cornflowerblue"])
for i, color in zip(range(n_classes), colors):
    plt.plot(
        fpr[i],
        tpr[i],
        color=color,
        lw=lw,
        label="ROC curve of class {0} (area = {1:0.2f})".format(i, roc_auc[i]),
    )

plt.plot([0, 1], [0, 1], "k--", lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Some extension of Receiver operating characteristic to multiclass")
plt.legend(loc="lower right")
plt.show()

MACROPMS = roc_auc["macro"]
MACROPMS0 = roc_auc[0]
MACROPMS1 = roc_auc[1]
MACROPMS2 = roc_auc[2]
MACROPMS3 = roc_auc[3]
MACROPMS4 = roc_auc[4]
MACROPMS5 = roc_auc[5]

print(MACROPMS)


### 'PLS'

In [ ]:
from keras import models
from tensorflow.keras.optimizers import Adam


X = df.loc[:,['FLEP8','FLEP4','WHP','BP',
       'StreamEuc', 'Slope', 'RoadEuc', 'StrucEuc',
        'DEM', 'PRS','PMS']]

y = df.iloc[:,24:25].values
y_lab = df.iloc[:,24].values


from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0,1))
X = sc.fit_transform(X)

from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
y = ohe.fit_transform(y).toarray()

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.33)

scores = model.evaluate(X_test, y_test, verbose=False)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


model = Sequential()
#model.add(Dense(64, activation = "relu"))
#model.add(Dense(64, activation = "sigmoid"))
model.add(Dense(64, activation = "relu"))
#model.add(Dense(64, activation = "relu"))
model.add(Dense(6, activation = "softmax"))
model.compile(loss = "binary_crossentropy",
             optimizer = Adam(learning_rate=0.001),
             metrics = ["accuracy"])

history = model.fit(X_train, y_train, epochs = 1, batch_size = 64, validation_data=(X_test, y_test))



y_val_cat_prob=model.predict(X_test)
n_classes=6
import numpy as np
import matplotlib.pyplot as plt
from itertools import cycle
from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_auc_score

fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve( y_test[:, i] , y_val_cat_prob[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])    
    
lw=.8
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])

mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

plt.figure()

plt.plot(
    fpr["macro"],
    tpr["macro"],
    label="macro-average ROC curve (area = {0:0.2f})".format(roc_auc["macro"]),
    color="navy",
    linestyle=":",
    linewidth=4,
)

label = ["1","2","3","4","5","6"]

colors = cycle(["aqua", "darkorange", "cornflowerblue"])
for i, color in zip(range(n_classes), colors):
    plt.plot(
        fpr[i],
        tpr[i],
        color=color,
        lw=lw,
        label="ROC curve of class {0} (area = {1:0.2f})".format(i, roc_auc[i]),
        #label = label[i]
    )

plt.plot([0, 1], [0, 1], "k--", lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Some extension of Receiver operating characteristic to multiclass")
plt.legend(loc="lower right")
plt.show()

MACROPLS = roc_auc["macro"]
MACROPLS0 = roc_auc[0]
MACROPLS1 = roc_auc[1]
MACROPLS2 = roc_auc[2]
MACROPLS3 = roc_auc[3]
MACROPLS4 = roc_auc[4]
MACROPLS5 = roc_auc[5]

print(MACROPLS)


### Variable Importance 

In [ ]:
list = [MACROFULL-MACROFLEP8 , MACROFULL-MACROFLEP4 , MACROFULL-MACROWHP , MACROFULL-MACROBP , MACROFULL-MACROStreamEuc , MACROFULL-MACROSlope , MACROFULL-MACROStrucEuc, MACROFULL-MACRORoadEuc, MACROFULL-MACRODEM , MACROFULL-MACROPRS , MACROFULL-MACROPMS, MACROFULL-MACROPLS]
list0 = [MACROFULL0-MACROFLEP80 , MACROFULL0-MACROFLEP40 , MACROFULL0-MACROWHP0 , MACROFULL0-MACROBP0 , MACROFULL0-MACROStreamEuc0 , MACROFULL0-MACROSlope0 , MACROFULL0-MACROStrucEuc0 ,MACROFULL0-MACRORoadEuc0 , MACROFULL0-MACRODEM0 , MACROFULL0-MACROPRS0 , MACROFULL0-MACROPMS0, MACROFULL0-MACROPLS0]
list1 = [MACROFULL1-MACROFLEP81 , MACROFULL1-MACROFLEP41 , MACROFULL1-MACROWHP1 , MACROFULL1-MACROBP1 , MACROFULL1-MACROStreamEuc1 , MACROFULL1-MACROSlope1 , MACROFULL1-MACROStrucEuc1 ,MACROFULL1-MACRORoadEuc1 , MACROFULL1-MACRODEM1 , MACROFULL1-MACROPRS1 , MACROFULL1-MACROPMS1, MACROFULL1-MACROPLS1]
list2 = [MACROFULL2-MACROFLEP82 , MACROFULL2-MACROFLEP42 , MACROFULL2-MACROWHP2 , MACROFULL2-MACROBP2 , MACROFULL2-MACROStreamEuc2 , MACROFULL2-MACROSlope2 , MACROFULL2-MACROStrucEuc2 ,MACROFULL2-MACRORoadEuc2 , MACROFULL2-MACRODEM2 , MACROFULL2-MACROPRS2 , MACROFULL2-MACROPMS2, MACROFULL2-MACROPLS2]
list3 = [MACROFULL3-MACROFLEP83 , MACROFULL3-MACROFLEP43 , MACROFULL3-MACROWHP3 , MACROFULL3-MACROBP3 , MACROFULL3-MACROStreamEuc3 , MACROFULL3-MACROSlope3 , MACROFULL3-MACROStrucEuc3 ,MACROFULL3-MACRORoadEuc3 , MACROFULL3-MACRODEM3 , MACROFULL3-MACROPRS3 , MACROFULL3-MACROPMS3, MACROFULL3-MACROPLS3]
list4 = [MACROFULL4-MACROFLEP84 , MACROFULL4-MACROFLEP44 , MACROFULL4-MACROWHP4 , MACROFULL4-MACROBP4 , MACROFULL4-MACROStreamEuc4 , MACROFULL4-MACROSlope4 , MACROFULL4-MACROStrucEuc4 ,MACROFULL4-MACRORoadEuc4 , MACROFULL4-MACRODEM4 , MACROFULL4-MACROPRS4 , MACROFULL4-MACROPMS4, MACROFULL4-MACROPLS4]
list5 = [MACROFULL5-MACROFLEP85 , MACROFULL5-MACROFLEP45 , MACROFULL5-MACROWHP5 , MACROFULL5-MACROBP5 , MACROFULL5-MACROStreamEuc5 , MACROFULL5-MACROSlope5 , MACROFULL5-MACROStrucEuc5 ,MACROFULL5-MACRORoadEuc5 , MACROFULL5-MACRODEM5 , MACROFULL5-MACROPRS5 , MACROFULL5-MACROPMS5, MACROFULL5-MACROPLS5]


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
 
barWidth = 0.1
fig = plt.subplots(figsize =(16, 8))
 
br1 = np.arange(len(list))
br2 = [x + barWidth for x in br1]
br3 = [x + barWidth for x in br2]
br4 = [x + barWidth for x in br3]
br5 = [x + barWidth for x in br4]
br6 = [x + barWidth for x in br5]
br7 = [x + barWidth for x in br6]

plt.bar(br1, list, color ='black', width = barWidth,
        edgecolor ='black', label ='Macro Average')
plt.bar(br2, list0, color ='dimgrey', width = barWidth,
        edgecolor ='black', label ='Broadcast Burning')
plt.bar(br3, list1, color ='grey', width = barWidth,
        edgecolor ='black', label ='Burning of Piled Material')
plt.bar(br4, list2, color ='darkgrey', width = barWidth,
        edgecolor ='black', label ='Piling of Fuels')
plt.bar(br5, list3, color ='lightgray', width = barWidth,
        edgecolor ='black', label ='Yarding')
plt.bar(br6, list4, color ='gainsboro', width = barWidth,
        edgecolor ='black', label ='Thinning for HFR')
plt.bar(br7, list5, color ='white', width = barWidth,
        edgecolor ='black', label ='Multiple Treatments')
 
# Adding Xticks
plt.xlabel('Variable', fontweight ='bold', fontsize = 18)
plt.ylabel('Variable Importance', fontweight ='bold', fontsize = 18)
plt.ylim([0, 0.04])
plt.legend(loc='upper left',facecolor='white', framealpha=1, fontsize=15)
plt.xticks([r + barWidth for r in range(len(list))],
        ["FLEP8" , "FLEP4" , "WHP" , "BP" , "StreamEuc" , "Slope" , "StrucEuc", "RoadEuc" , "DEM" , "PRS" , "PMS", "PLS"], rotation=90, fontsize=18
)
plt.yticks(fontsize=19)
 
plt.show()

In [ ]:
outputfilelocation = "C:\\Users\\Desktop\\HFT\\HFT_ForPrediction_90m_PRED_MASK_NoTreatment.txt"

dfEND3.to_csv(outputfilelocation, 
          header=True, 
          index=None, 
          sep=',', 
          mode='a')

In [ ]:
datadummy = { 'OID_':[1,2],
             'pointid':[1,2],
             'grid_code':[1,2],
             'WHP':[0,7817],
             'RPS':[0,1.178],
             'NatForest':[1,2],
             'FLEP8':[0,1],
             'FLEP4':[0,1],
             'CRPS':[0,99.8465195],
             'CFL':[0,35.4152641 ],
             'BP':[0,0.0133],
             'StreamEuc':[0,4590.392],
             'Slope':[0,73],
             'RoadEuc':[0,13165.5917969],
             'DEM':[1623,4387],
             'Aspect':[-1,359.72],
             'StrucEuc':[0,12844.09],
             'NVC': [1,2],
             'PRS': [2,20],
             'SCLASS': [1,2],
             'VCC': [1,2],
             'PMS':[1,16],
             'PLS':[1,18],
             'FRG':[1,2],
             'ACTIVITY_C':[1,2]}

### ModelPrediction 

In [ ]:
import pandas as pd

X = dfPrediction.loc[:, ['FLEP8','WHP','BP',
       'StreamEuc', 'Slope', 'RoadEuc', 'StrucEuc']]

from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0,1))
X = sc.fit_transform(X)

In [ ]:
import numpy as np

ynew = model.predict(X)
col1 = ynew[:,0]
col2 = ynew[:,1]
col3 = ynew[:,2]
col4 = ynew[:,3]
col5 = ynew[:,4]
col6 = ynew[:,5]
col7 = np.argmax(ynew, axis = 1)[:]

dfPrediction['pred1'] = col1.tolist()
dfPrediction['pred2'] = col2.tolist()
dfPrediction['pred3'] = col3.tolist()
dfPrediction['pred4'] = col4.tolist()
dfPrediction['pred5'] = col5.tolist()
dfPrediction['pred6'] = col6.tolist()
dfPrediction['predMAX'] = col7.tolist()


In [ ]:
dfpred2 = dfPrediction.loc[:, ['Lat',
       'Long', 'pred1', 'pred2', 'pred3', 'pred4', 'pred5','pred6','predMAX']]

In [ ]:
outputfilelocation = "C:\\Users\\Desktop\\HFT\\HFT_ForPrediction_90m_PRED.txt"

dfpred2.to_csv(outputfilelocation, 
          header=True, 
          index=None, 
          sep=',', 
          mode='a')

### CROSS VALIDATE BY GRID CODE

In [ ]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0,1))


from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()

X = df.loc[:, ['FLEP8', 'FLEP4','WHP','BP',
       'StreamEuc', 'Slope', 'RoadEuc', 'StrucEuc',
        'DEM', 'PRS','PMS','PLS','grid_code','ACTIVITY_C']]#, 'CO_Aspect','ACTIVITY_C','grid_code'
XT = sc.fit_transform(X)

yALL = X.iloc[:,13:14].values
yALL_LAB = X.iloc[:,13].values

YALL = ohe.fit_transform(yALL).toarray()

dfT = pd.DataFrame(XT, columns =['FLEP8', 'FLEP4','WHP','BP',
       'StreamEuc', 'Slope', 'RoadEuc', 'StrucEuc',
        'DEM', 'PRS','PMS','PLS','grid_code','ACTIVITY_C'])

dfT["grid_code"] = round(dfT["grid_code"],2)

dfG1 = dfT[dfT["grid_code"]==0]
dfG2 = dfT[dfT["grid_code"]==0.14]
dfG3 = dfT[dfT["grid_code"]==0.29]
dfG4 = dfT[dfT["grid_code"]==0.43]
dfG5 = dfT[dfT["grid_code"]==0.57]
dfG6 = dfT[dfT["grid_code"]==0.71]
dfG7 = dfT[dfT["grid_code"]==0.86]
dfG8 = dfT[dfT["grid_code"]==1]


XG1 = dfG1.loc[:, ['FLEP8', 'FLEP4','WHP','BP',
       'StreamEuc', 'Slope', 'RoadEuc', 'StrucEuc',
        'DEM', 'PRS','PMS','PLS','ACTIVITY_C']]
XG2 = dfG2.loc[:, ['FLEP8', 'FLEP4','WHP','BP',
       'StreamEuc', 'Slope', 'RoadEuc', 'StrucEuc',
        'DEM', 'PRS','PMS','PLS','ACTIVITY_C']]
XG3 = dfG3.loc[:, ['FLEP8', 'FLEP4','WHP','BP',
       'StreamEuc', 'Slope', 'RoadEuc', 'StrucEuc',
        'DEM', 'PRS','PMS','PLS','ACTIVITY_C']]
XG4 = dfG4.loc[:, ['FLEP8', 'FLEP4','WHP','BP',
       'StreamEuc', 'Slope', 'RoadEuc', 'StrucEuc',
        'DEM', 'PRS','PMS','PLS','ACTIVITY_C']]
XG5 = dfG5.loc[:, ['FLEP8', 'FLEP4','WHP','BP',
       'StreamEuc', 'Slope', 'RoadEuc', 'StrucEuc',
        'DEM', 'PRS','PMS','PLS','ACTIVITY_C']]
XG6 = dfG6.loc[:, ['FLEP8', 'FLEP4','WHP','BP',
       'StreamEuc', 'Slope', 'RoadEuc', 'StrucEuc',
        'DEM', 'PRS','PMS','PLS','ACTIVITY_C']]
XG7 = dfG7.loc[:, ['FLEP8', 'FLEP4','WHP','BP',
       'StreamEuc', 'Slope', 'RoadEuc', 'StrucEuc',
        'DEM', 'PRS','PMS','PLS','ACTIVITY_C']]
XG8 = dfG8.loc[:, ['FLEP8', 'FLEP4','WHP','BP',
       'StreamEuc', 'Slope', 'RoadEuc', 'StrucEuc',
        'DEM', 'PRS','PMS','PLS','ACTIVITY_C']]


y1 = XG1.iloc[:,12:13].values
y_lab1 = XG1.iloc[:,12].values
y2 = XG2.iloc[:,12:13].values
y_lab2 = XG2.iloc[:,12].values
y3 = XG3.iloc[:,12:13].values
y_lab3 = XG3.iloc[:,12].values
y4 = XG4.iloc[:,12:13].values
y_lab4 = XG4.iloc[:,12].values

y5 = XG5.iloc[:,12:13].values
y_lab5 = XG5.iloc[:,12].values
y6 = XG6.iloc[:,12:13].values
y_lab6 = XG6.iloc[:,12].values
y7 = XG7.iloc[:,12:13].values
y_lab7 = XG7.iloc[:,12].values
y8 = XG8.iloc[:,12:13].values
y_lab8 = XG8.iloc[:,12].values



y1_OHE = ohe.fit_transform(y1).toarray()
y2_OHE = ohe.fit_transform(y2).toarray()
y3_OHE = ohe.fit_transform(y3).toarray()
y4_OHE = ohe.fit_transform(y4).toarray()
y5_OHE = ohe.fit_transform(y5).toarray()
y6_OHE = ohe.fit_transform(y6).toarray()
y7_OHE = ohe.fit_transform(y7).toarray()
y8_OHE = ohe.fit_transform(y8).toarray()


X1 = XG1.loc[:, ['FLEP8', 'FLEP4','WHP','BP',
       'StreamEuc', 'Slope', 'RoadEuc', 'StrucEuc',
        'DEM', 'PRS','PMS','PLS']]

X2 = XG2.loc[:, ['FLEP8', 'FLEP4','WHP','BP',
       'StreamEuc', 'Slope', 'RoadEuc', 'StrucEuc',
        'DEM', 'PRS','PMS','PLS']]
X3 = XG3.loc[:, ['FLEP8', 'FLEP4','WHP','BP',
       'StreamEuc', 'Slope', 'RoadEuc', 'StrucEuc',
        'DEM', 'PRS','PMS','PLS']]
X4 = XG4.loc[:, ['FLEP8', 'FLEP4','WHP','BP',
       'StreamEuc', 'Slope', 'RoadEuc', 'StrucEuc',
        'DEM', 'PRS','PMS','PLS']]
X5 = XG5.loc[:, ['FLEP8', 'FLEP4','WHP','BP',
       'StreamEuc', 'Slope', 'RoadEuc', 'StrucEuc',
        'DEM', 'PRS','PMS','PLS']]
X6 = XG6.loc[:, ['FLEP8', 'FLEP4','WHP','BP',
       'StreamEuc', 'Slope', 'RoadEuc', 'StrucEuc',
        'DEM', 'PRS','PMS','PLS']]
X7 = XG7.loc[:, ['FLEP8', 'FLEP4','WHP','BP',
       'StreamEuc', 'Slope', 'RoadEuc', 'StrucEuc',
        'DEM', 'PRS','PMS','PLS']]
X8 = XG8.loc[:, ['FLEP8', 'FLEP4','WHP','BP',
       'StreamEuc', 'Slope', 'RoadEuc', 'StrucEuc',
        'DEM', 'PRS','PMS','PLS']]

X1 = sc.fit_transform(X1)
X2 = sc.fit_transform(X2)
X3 = sc.fit_transform(X3)
X4 = sc.fit_transform(X4)
X5 = sc.fit_transform(X5)
X6 = sc.fit_transform(X6)
X7 = sc.fit_transform(X7)
X8 = sc.fit_transform(X8)





from sklearn.model_selection import train_test_split
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1,y1_OHE,test_size = 0.33)
from sklearn.model_selection import train_test_split
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2,y2_OHE,test_size = 0.33)
from sklearn.model_selection import train_test_split
X_train3, X_test3, y_train3, y_test3 = train_test_split(X3,y3_OHE,test_size = 0.33)
from sklearn.model_selection import train_test_split
X_train4, X_test4, y_train4, y_test4 = train_test_split(X4,y4_OHE,test_size = 0.33)
from sklearn.model_selection import train_test_split
X_train5, X_test5, y_train5, y_test5 = train_test_split(X5,y5_OHE,test_size = 0.33)
from sklearn.model_selection import train_test_split
X_train6, X_test6, y_train6, y_test6 = train_test_split(X6,y6_OHE,test_size = 0.33)
from sklearn.model_selection import train_test_split
X_train7, X_test7, y_train7, y_test7 = train_test_split(X7,y7_OHE,test_size = 0.33)
from sklearn.model_selection import train_test_split
X_train8, X_test8, y_train8, y_test8 = train_test_split(X8,y8_OHE,test_size = 0.33)



In [ ]:
df["one"]=1

In [ ]:
import numpy as np
unique_elements, counts_elements = np.unique(dfG2["ACTIVITY_C"], return_counts=True)
print("Frequency of unique values of the said array:")
print(np.asarray((unique_elements, counts_elements)))
df.groupby(['ACTIVITY_C','grid_code'])['one'].count().reset_index()

# 1 - Manti-La Sal National Forest
# 2 - Rio Grande National Forest
# 3 - San Juan National Forest
# 4 - Arapaho and Roosevelt National Forests
# 5 - Grand Mesa, Uncompahgre, and Gunnison National Forests
# 6 - White River National Forest
# 7 - Medicine Bow-Routt National Forest
# 8 - Pike and San Isabel National Forests


In [ ]:
## ROC AUC

y_val_cat_prob=model.predict(X2)
n_classes=6
# Compute ROC curve and ROC area for each class
import numpy as np
import matplotlib.pyplot as plt
from itertools import cycle
from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_auc_score

fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve( y2_OHE[:, i] , y_val_cat_prob[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])    

lw=.8
# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# Plot all ROC curves
plt.figure()


plt.plot(
    fpr["macro"],
    tpr["macro"],
    label="macro-average ROC curve (area = {0:0.2f})".format(roc_auc["macro"]),
    color="navy",
    linestyle=":",
    linewidth=4,
)

label = ["1","2","3","4","5"]

colors = cycle(["aqua", "darkorange", "cornflowerblue"])
for i, color in zip(range(n_classes), colors):
    plt.plot(
        fpr[i],
        tpr[i],
        color=color,
        lw=lw,
        label="ROC curve of class {0} (area = {1:0.2f})".format(i, roc_auc[i]),
        #label = label[i]
    )

plt.plot([0, 1], [0, 1], "k--", lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Some extension of Receiver operating characteristic to multiclass")
plt.legend(loc="lower right")
plt.show()

MACROFULL_G2 = roc_auc["macro"]
MACROFULL0_G2 = roc_auc[0]
MACROFULL1_G2 = roc_auc[1]
MACROFULL2_G2 = roc_auc[2]
MACROFULL3_G2 = roc_auc[3]
MACROFULL4_G2 = roc_auc[4]
MACROFULL5_G2 = roc_auc[5]

print(MACROFULL_G2)

In [ ]:
## ROC AUC

y_val_cat_prob=model.predict(X3)
n_classes=6
# Compute ROC curve and ROC area for each class
import numpy as np
import matplotlib.pyplot as plt
from itertools import cycle
from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_auc_score

fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve( y3_OHE[:, i] , y_val_cat_prob[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])    

lw=.8
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])

mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# Plot all ROC curves
plt.figure()


plt.plot(
    fpr["macro"],
    tpr["macro"],
    label="macro-average ROC curve (area = {0:0.2f})".format(roc_auc["macro"]),
    color="navy",
    linestyle=":",
    linewidth=4,
)

label = ["1","2","3","4","5"]

colors = cycle(["aqua", "darkorange", "cornflowerblue"])
for i, color in zip(range(n_classes), colors):
    plt.plot(
        fpr[i],
        tpr[i],
        color=color,
        lw=lw,
        label="ROC curve of class {0} (area = {1:0.2f})".format(i, roc_auc[i]),
        #label = label[i]
    )

plt.plot([0, 1], [0, 1], "k--", lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Some extension of Receiver operating characteristic to multiclass")
plt.legend(loc="lower right")
plt.show()

MACROFULL_G3 = roc_auc["macro"]
MACROFULL0_G3 = roc_auc[0]
MACROFULL1_G3 = roc_auc[1]
MACROFULL2_G3 = roc_auc[2]
MACROFULL3_G3 = roc_auc[3]
MACROFULL4_G3 = roc_auc[4]
MACROFULL5_G3 = roc_auc[5]

print(MACROFULL_G3)

In [ ]:
## ROC AUC

y_val_cat_prob=model.predict(X4)
n_classes=6
# Compute ROC curve and ROC area for each class
import numpy as np
import matplotlib.pyplot as plt
from itertools import cycle
from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_auc_score

fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve( y4_OHE[:, i] , y_val_cat_prob[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])    
    
lw=.8
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# Plot all ROC curves
plt.figure()


plt.plot(
    fpr["macro"],
    tpr["macro"],
    label="macro-average ROC curve (area = {0:0.2f})".format(roc_auc["macro"]),
    color="navy",
    linestyle=":",
    linewidth=4,
)

label = ["1","2","3","4","5"]

colors = cycle(["aqua", "darkorange", "cornflowerblue"])
for i, color in zip(range(n_classes), colors):
    plt.plot(
        fpr[i],
        tpr[i],
        color=color,
        lw=lw,
        label="ROC curve of class {0} (area = {1:0.2f})".format(i, roc_auc[i]),
        #label = label[i]
    )

plt.plot([0, 1], [0, 1], "k--", lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Some extension of Receiver operating characteristic to multiclass")
plt.legend(loc="lower right")
plt.show()

MACROFULL_G4 = roc_auc["macro"]
MACROFULL0_G4 = roc_auc[0]
MACROFULL1_G4 = roc_auc[1]
MACROFULL2_G4 = roc_auc[2]
MACROFULL3_G4 = roc_auc[3]
MACROFULL4_G4 = roc_auc[4]
MACROFULL5_G4 = roc_auc[5]


print(MACROFULL_G4)

In [ ]:
## ROC AUC

y_val_cat_prob=model.predict(X5)
n_classes=6
import numpy as np
import matplotlib.pyplot as plt
from itertools import cycle
from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_auc_score

fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve( y5_OHE[:, i] , y_val_cat_prob[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])    
    

lw=.8
# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

plt.figure()


plt.plot(
    fpr["macro"],
    tpr["macro"],
    label="macro-average ROC curve (area = {0:0.2f})".format(roc_auc["macro"]),
    color="navy",
    linestyle=":",
    linewidth=4,
)

label = ["1","2","3","4","5"]

colors = cycle(["aqua", "darkorange", "cornflowerblue"])
for i, color in zip(range(n_classes), colors):
    plt.plot(
        fpr[i],
        tpr[i],
        color=color,
        lw=lw,
        label="ROC curve of class {0} (area = {1:0.2f})".format(i, roc_auc[i]),
        #label = label[i]
    )

plt.plot([0, 1], [0, 1], "k--", lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Some extension of Receiver operating characteristic to multiclass")
plt.legend(loc="lower right")
plt.show()

MACROFULL_G5 = roc_auc["macro"]
MACROFULL0_G5 = roc_auc[0]
MACROFULL1_G5 = roc_auc[1]
MACROFULL2_G5 = roc_auc[2]
MACROFULL3_G5 = roc_auc[3]
MACROFULL4_G5 = roc_auc[4]
MACROFULL5_G5 = roc_auc[5]

print(MACROFULL_G5)

In [ ]:
## ROC AUC

y_val_cat_prob=model.predict(X6)
n_classes=6
# Compute ROC curve and ROC area for each class
import numpy as np
import matplotlib.pyplot as plt
from itertools import cycle
from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_auc_score

fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve( y6_OHE[:, i] , y_val_cat_prob[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])    
    

lw=.8
# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# Plot all ROC curves
plt.figure()

plt.plot(
    fpr["macro"],
    tpr["macro"],
    label="macro-average ROC curve (area = {0:0.2f})".format(roc_auc["macro"]),
    color="navy",
    linestyle=":",
    linewidth=4,
)

label = ["1","2","3","4","5"]

colors = cycle(["aqua", "darkorange", "cornflowerblue"])
for i, color in zip(range(n_classes), colors):
    plt.plot(
        fpr[i],
        tpr[i],
        color=color,
        lw=lw,
        label="ROC curve of class {0} (area = {1:0.2f})".format(i, roc_auc[i]),
        #label = label[i]
    )

plt.plot([0, 1], [0, 1], "k--", lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Some extension of Receiver operating characteristic to multiclass")
plt.legend(loc="lower right")
plt.show()

MACROFULL_G6 = roc_auc["macro"]
MACROFULL0_G6 = roc_auc[0]
MACROFULL1_G6 = roc_auc[1]
MACROFULL2_G6 = roc_auc[2]
MACROFULL3_G6 = roc_auc[3]
MACROFULL4_G6 = roc_auc[4]
MACROFULL5_G6 = roc_auc[5]


print(MACROFULL_G6)

In [ ]:


## ROC AUC

y_val_cat_prob=model.predict(X7)
n_classes=6
# Compute ROC curve and ROC area for each class
import numpy as np
import matplotlib.pyplot as plt
from itertools import cycle
from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_auc_score

fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve( y7_OHE[:, i] , y_val_cat_prob[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])    
    

lw=.8
# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# Plot all ROC curves
plt.figure()


plt.plot(
    fpr["macro"],
    tpr["macro"],
    label="macro-average ROC curve (area = {0:0.2f})".format(roc_auc["macro"]),
    color="navy",
    linestyle=":",
    linewidth=4,
)

label = ["1","2","3","4","5"]

colors = cycle(["aqua", "darkorange", "cornflowerblue"])
for i, color in zip(range(n_classes), colors):
    plt.plot(
        fpr[i],
        tpr[i],
        color=color,
        lw=lw,
        label="ROC curve of class {0} (area = {1:0.2f})".format(i, roc_auc[i]),
        #label = label[i]
    )

plt.plot([0, 1], [0, 1], "k--", lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Some extension of Receiver operating characteristic to multiclass")
plt.legend(loc="lower right")
plt.show()

MACROFULL_G7 = roc_auc["macro"]
MACROFULL0_G7 = roc_auc[0]
MACROFULL1_G7 = roc_auc[1]
MACROFULL2_G7 = roc_auc[2]
MACROFULL3_G7 = roc_auc[3]
MACROFULL4_G7 = roc_auc[4]
MACROFULL5_G7 = roc_auc[5]


print(MACROFULL_G7)

In [ ]:


## ROC AUC

y_val_cat_prob=model.predict(X8)
n_classes=6
# Compute ROC curve and ROC area for each class
import numpy as np
import matplotlib.pyplot as plt
from itertools import cycle
from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_auc_score

fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve( y8_OHE[:, i] , y_val_cat_prob[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])    
    
lw=.8
# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# Plot all ROC curves
plt.figure()

plt.plot(
    fpr["macro"],
    tpr["macro"],
    label="macro-average ROC curve (area = {0:0.2f})".format(roc_auc["macro"]),
    color="navy",
    linestyle=":",
    linewidth=4,
)

label = ["1","2","3","4","5"]

colors = cycle(["aqua", "darkorange", "cornflowerblue"])
for i, color in zip(range(n_classes), colors):
    plt.plot(
        fpr[i],
        tpr[i],
        color=color,
        lw=lw,
        label="ROC curve of class {0} (area = {1:0.2f})".format(i, roc_auc[i]),
        #label = label[i]
    )

plt.plot([0, 1], [0, 1], "k--", lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Some extension of Receiver operating characteristic to multiclass")
plt.legend(loc="lower right")
plt.show()

MACROFULL_G8 = roc_auc["macro"]
MACROFULL0_G8 = roc_auc[0]
MACROFULL1_G8 = roc_auc[1]
MACROFULL2_G8 = roc_auc[2]
MACROFULL3_G8 = roc_auc[3]
MACROFULL4_G8 = roc_auc[4]
MACROFULL5_G8 = roc_auc[5]


print(MACROFULL_G8)

In [ ]:
list = [MACROFULL_G2 , MACROFULL_G3 , MACROFULL_G4 , MACROFULL_G5 , MACROFULL_G6 , MACROFULL_G7 , MACROFULL_G8]
list0 = [MACROFULL0_G2 , MACROFULL0_G3 , MACROFULL0_G4 , MACROFULL0_G5 , MACROFULL0_G6 , MACROFULL0_G7 , MACROFULL0_G8]
list1 = [MACROFULL1_G2 , MACROFULL1_G3 , MACROFULL1_G4 , MACROFULL1_G5 , MACROFULL1_G6 , MACROFULL1_G7 , MACROFULL1_G8]
list2 = [MACROFULL2_G2 , MACROFULL2_G3 , MACROFULL2_G4 , MACROFULL2_G5 , MACROFULL2_G6 , MACROFULL2_G7 , MACROFULL2_G8]
list3 = [MACROFULL3_G2 , MACROFULL3_G3 , MACROFULL3_G4 , MACROFULL3_G5 , MACROFULL3_G6 , MACROFULL3_G7 , MACROFULL3_G8]
list4 = [MACROFULL4_G2 , MACROFULL4_G3 , MACROFULL4_G4 , MACROFULL4_G5 , MACROFULL4_G6 , MACROFULL4_G7 , MACROFULL4_G8]
list5 = [MACROFULL5_G2 , MACROFULL5_G3 , MACROFULL5_G4 , MACROFULL5_G5 , MACROFULL5_G6 , MACROFULL5_G7 , MACROFULL5_G8]


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
 
barWidth = 0.1
fig = plt.subplots(figsize =(16, 8))

br1 = np.arange(len(list))
br2 = [x + barWidth for x in br1]
br3 = [x + barWidth for x in br2]
br4 = [x + barWidth for x in br3]
br5 = [x + barWidth for x in br4]
br6 = [x + barWidth for x in br5]
br7 = [x + barWidth for x in br6]


plt.bar(br1, list, color ='black', width = barWidth,
        edgecolor ='black', label ='Macro Average')
plt.bar(br2, list0, color ='dimgrey', width = barWidth,
        edgecolor ='black', label ='Broadcast Burning')
plt.bar(br3, list1, color ='grey', width = barWidth,
        edgecolor ='black', label ='Burning of Piled Material')
plt.bar(br4, list2, color ='darkgrey', width = barWidth,
        edgecolor ='black', label ='Piling of Fuels')
plt.bar(br5, list3, color ='lightgray', width = barWidth,
        edgecolor ='black', label ='Yarding')
plt.bar(br6, list4, color ='gainsboro', width = barWidth,
        edgecolor ='black', label ='Thinning for HFR')
plt.bar(br7, list5, color ='white', width = barWidth,
        edgecolor ='black', label ='Multiple treatments')
 
plt.xlabel('Forest Region', fontweight ='bold', fontsize = 18)
plt.ylabel('AUC', fontweight ='bold', fontsize = 18)
plt.ylim([0, 1])
plt.legend(loc='lower right',facecolor='white', framealpha=1, fontsize=14)
plt.xticks([r + barWidth for r in range(len(list))],
        ["Rio Grande" , "San Juan" , "Arapaho\n and Roosevelt" , "Grand Mesa,\n Uncompahgre,\n and Gunnison" , "White River" , "Medicine Bow-Routt ","Pike\n and San Isabel"], rotation=90, fontsize=18
)
plt.yticks(fontsize=19)

plt.savefig("C:\\Users\\Desktop\\HFT\\Figures\\AUC_ForestCROSSVALIDATE.png", transparent=False,dpi=300, bbox_inches = "tight",facecolor='white')#
plt.show()